In [2]:
import sys
import os
import platform
from typing import List
import hashlib

def find_pdfs_based_on_os() -> List[str]:
    """
    Find all unique PDF files based on content, considering the operating system.
    Returns a list of paths to the unique PDF files.
    """
    def handle_os_error(err):
        print(f"OS error: {err}", file=sys.stderr)

    def get_start_directories() -> List[str]:
        """ Return appropriate start directories based on the operating system. """
        os_type = platform.system()
        if os_type == 'Windows':
            base_dir = os.environ.get('USERPROFILE', 'C:\\Users')
            return [os.path.join(base_dir, 'Documents')]
        elif os_type == 'Darwin':  # macOS
            return ['/Users']
        return ['/']

    def hash_file(filepath: str) -> str:
        """ Generate a SHA-256 hash of a file's contents. """
        hasher = hashlib.sha256()
        try:
            with open(filepath, 'rb') as f:
                buffer = f.read(65536)  # Read the file in chunks (e.g., 64kB)
                while len(buffer) > 0:
                    hasher.update(buffer)
                    buffer = f.read(65536)
        except IOError as e:
            print(f"Error reading file {filepath}: {str(e)}", file=sys.stderr)
            return None
        return hasher.hexdigest()

    def find_pdfs(directory: str, seen_hashes: set) -> List[str]:
        """ Recursively find all unique PDF files in the specified directory, handling permissions gracefully. """
        pdf_files = []
        for root, dirs, files in os.walk(directory, onerror=handle_os_error):
            for file in files:
                if file.lower().endswith('.pdf'):
                    full_path = os.path.join(root, file)
                    file_hash = hash_file(full_path)
                    if file_hash and file_hash not in seen_hashes:
                        seen_hashes.add(file_hash)
                        pdf_files.append(full_path)
        return pdf_files

    seen_hashes = set()
    pdfs = []
    start_directories = get_start_directories()
    for directory in start_directories:
        print(f"Scanning {directory} for PDF files...")
        pdfs.extend(find_pdfs(directory, seen_hashes))
    
    return pdfs

# Usage
pdf_files = find_pdfs_based_on_os()
print(f"Found {len(pdf_files)} unique PDFs based on content.")
for pdf in pdf_files[:10]:  # Print the first 10 PDF paths
    print(pdf)

Scanning /Users for PDF files...


OS error: [Errno 13] Permission denied: '/Users/yecao/.config/truffle'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Pictures/Photos Library.photoslibrary'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/CallHistoryTransactions'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/CloudDocs/session/db'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/com.apple.sharedfilelist'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/Knowledge'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/com.apple.TCC'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/FileProvider'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/AddressBook'
OS error: [Errno 1] Operation not permitted: '/Users/yecao/Library/Application Support/FaceTime'

Found 605 unique PDFs based on content.
/Users/yecao/Library/Application Support/pocketllm/user_workspace_cache/27fcc3e5-7bef-4a16-989e-eb38b3c9e5e6/model.ndb/documents/135/Certifier Training Handouts.pdf
/Users/yecao/Library/Application Support/pocketllm/user_workspace_cache/27fcc3e5-7bef-4a16-989e-eb38b3c9e5e6/model.ndb/documents/307/Position Update Process.pdf
/Users/yecao/Library/Application Support/pocketllm/user_workspace_cache/27fcc3e5-7bef-4a16-989e-eb38b3c9e5e6/model.ndb/documents/61/Check I-9 Status.pdf
/Users/yecao/Library/Application Support/pocketllm/user_workspace_cache/27fcc3e5-7bef-4a16-989e-eb38b3c9e5e6/model.ndb/documents/95/Finance Dashboard View Fund Balances.pdf
/Users/yecao/Library/Application Support/pocketllm/user_workspace_cache/27fcc3e5-7bef-4a16-989e-eb38b3c9e5e6/model.ndb/documents/300/Cómo ingresar horas en Web Clock.pdf
/Users/yecao/Library/Application Support/pocketllm/user_workspace_cache/27fcc3e5-7bef-4a16-989e-eb38b3c9e5e6/model.ndb/documents/132/SPFF

In [17]:
import hashlib
import os
from typing import Dict

class FileIdentity:
    def __init__(self, file_path: str, file_content_hash: str, combined_hash: str):
        self.file_path = file_path
        self.file_content_hash = file_content_hash
        self.combined_hash = combined_hash

class FileManager:
    def __init__(self, root_directory: str):
        self.root_directory = root_directory
        self.file_identities: Dict[str, FileIdentity] = {}

    def scan_files(self) -> Dict[str, FileIdentity]:
        """Scan the directory for PDF files and return a dictionary of new file identities."""
        new_files = {}
        for root, _, files in os.walk(self.root_directory):
            for file in files:
                if file.endswith(".pdf"):
                    file_path = os.path.join(root, file)
                    content_hash = FileManager.hash_file_content(file_path)
                    combined_hash = FileManager.hash_combined(file_path, content_hash)
                    new_files[combined_hash] = FileIdentity(file_path, content_hash, combined_hash)
        return new_files

    def update_files(self):
        """Update the stored file identities based on the latest scan."""
        new_files = self.scan_files()
        added = set(new_files.keys()) - set(self.file_identities.keys())
        removed = set(self.file_identities.keys()) - set(new_files.keys())

        # Update the stored data
        for hash_key in removed:
            del self.file_identities[hash_key]
        self.file_identities.update(new_files)

        return {"added": list(added), "removed": list(removed)}

    @staticmethod
    def hash_file_content(file_path: str) -> str:
        """Generate a hash for the contents of a file."""
        with open(file_path, 'rb') as file:
            hasher = hashlib.sha256()
            while chunk := file.read(4096):
                hasher.update(chunk)
            return hasher.hexdigest()

    @staticmethod
    def hash_combined(file_path: str, content_hash: str) -> str:
        """Generate a combined hash of the file path and content hash."""
        return hashlib.sha256(f"{file_path}{content_hash}".encode()).hexdigest()

In [18]:
file_manager = FileManager(root_directory="/Users/yecao/Downloads")
file_manager.update_files()

{'added': ['8306376e37b5894ad28e9854f7725ee493ab569b4bf181462ae0c12c702f306c',
  '7f5154935499622232153dd4878d2b1fda1bc2bb1bab0bf93ba73b98de92078b',
  'c9593decae7ef4dd2ce86ec285fd94667b7a3556e6ad6128c68ddc0658d1f8e2',
  'e2f670fe738647a33e519bdd440bcd8313e98174dd684490c8dec69dffe889f2',
  '43723eafd8424343a40283a2e031c41085ec797570aa849b8fc0373449c9df6b',
  '60fd8f94597a300ec5dd187f023df41c6845e9ba6023f3c07eab8015624b9541',
  'fda81253ba4628f96fab3c8d697d572b0927eeb70b343a4220b197387cf12de2',
  '469dce342460a01b05c45df93872a207835589ec8368e6ba5ec5ffe6be0b7840',
  '485dd7b033b492d90038b73ee563cd91e49fa6c1bbe549351eab622d8053d037',
  '78f05b3771c50249ecee4e89b9a7d9f505031cf222bdb6c114532874f5886586',
  'd983a26aa486aea1b90635cf40fd1d4a380b7a078f1dbf79931905912e7560a9',
  '8261eb812f2a136e098eae086a21d267080aaa64f3cbfee35b09d327caff63fb',
  '1ed2543aaf4be5edcc270711042d803e460c2aaaaca9463148d3544c6cf0220d',
  '5445340ab66024b0fecd5af4fb8d5c884096dc1c91d8af9674167119105f30ec',
  '56146eca

In [20]:
changes = file_manager.update_files()
print(changes)

{'added': ['c0a4ddc420b197c254b4a0c708f743d497237a75ce56c0a7c5f1258ed0937d02'], 'removed': ['fda81253ba4628f96fab3c8d697d572b0927eeb70b343a4220b197387cf12de2']}


In [ ]:
# In your FastAPI application
from fastapi import FastAPI

app = FastAPI()
file_manager = FileManager("/path/to/search")

@app.post("/update-files/")
def update_files():
    changes = file_manager.update_files()
    return changes